## 디코더의 두번째 서브층: 인코더-디코더 어텐션

**참고문헌**  
\[1\] [16-01 트랜스포머(Transformer)](https://wikidocs.net/31379)  
\[2\] [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html#prelims)  

![fig1](../images/decoder._look_a_head.png)

디코더의 두번째 서브층에 대해서 이해해봅시다. 디코더의 두번째 서브층은 멀티 헤드 어텐션을 수행한다는  
점에서는 이전의 어텐션들(인코더와 디코더의 첫번째 서브층)과는 공통점이 있으나 이번에는 셀프 어텐션이 아님.  
<br>
셀프 어텐션은 Query, Key, Value가 같은 경우를 말하는데, 인코더-디코더 어텐션은 Query가 디코더인  
행렬인 반면, Key와 Value는 인코더 행렬이기 때문입니다. 다시 한 번 각 서브층에서의 Q,K,V의 관계를 정리해봄.  

|    layer             |                       |         
|----------------------|------------------------|
|인터더의 첫번째 서브층  | Query = **Key** = Value |
|디코더의 첫번째 서브층  | Query = **Key** = Value |
|디코더의 두번째 서브층  | Query : 디코더 행렬 /**Key** = Value : 인코더 행렬 |

디코더의 두번째 서브층을 확대해 보면, 다음과 같이 인코더로부터 두 개의 화살표가 그려져 있음.  

![fig_3](../images/decoder_2nd_layer.png)

두 개의 화살표는 각각  key와 value를 의미하며 이는 인코더의 마지막 층에서 온 행렬로부터 얻음.  
반면 Query는 더코더의 첫번째 서브층의 결과 행렬로부터 얻는다는 점이 다릅니다.  
Query가 디코더 행렬 Key가 인코더 행렬일 때, 어텐셜 스코어 행렬을 추구하는 과정은 다음과 같습니다.

![fig_4](../images/decoder_2nd_attention_score_matrix_final.png)